In [62]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
import os
import pandas as pd
import math
import numpy as np

Get sorted cases id

In [64]:
import wget
import os.path

In [87]:
df_final = pd.read_csv('Helpdesk2017_anonimyzed.csv')

In [88]:
df_final.columns

Index(['Case ID', 'Activity', 'Complete Timestamp', 'Resource'], dtype='object')

In [90]:
df_final.head()

,Case ID,Activity,Complete Timestamp,Resource
0,0,Assign seriousness,2012-10-09 14:50:17,Value 1
1,0,Take in charge ticket,2012-10-09 14:51:01,Value 1
2,0,Take in charge ticket,2012-10-12 15:02:56,Value 2
3,0,Resolve ticket,2012-10-25 11:54:26,Value 1
4,0,Closed,2012-11-09 12:54:39,Value 3


In [91]:
df_final['Complete Timestamp']

0        2012-10-09 14:50:17
1        2012-10-09 14:51:01
2        2012-10-12 15:02:56
3        2012-10-25 11:54:26
4        2012-11-09 12:54:39
                ...         
21343    2010-09-02 10:11:00
21344    2012-01-03 09:33:43
21345    2012-01-10 15:30:11
21346    2012-01-10 17:07:40
21347    2012-02-08 17:00:34
Name: Complete Timestamp, Length: 21348, dtype: object

In [92]:
sorted_time_cases = df_final.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values

In [93]:
df_final.shape

(21348, 4)

In [94]:
df_final.columns

Index(['Case ID', 'Activity', 'Complete Timestamp', 'Resource'], dtype='object')

In [95]:
df_final['Activity'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Insert ticket', 'Wait', 'Create SW anomaly',
       'Require upgrade', 'VERIFIED', 'DUPLICATE', 'Resolve SW anomaly',
       'Schedule intervention', 'RESOLVED', 'INVALID'], dtype=object)

Filter for `Closed` process

In [96]:
just_closed = True

if(just_closed):
    closed_cases = []

    for name, group in df_final.groupby('Case ID'):
        last_step = group.tail(1)['Activity'].values[0]
        if(last_step == 'Closed'):
            closed_cases.append(group['Case ID'].values[0])

    df_final = df_final[df_final['Case ID'].isin(closed_cases)]     

Example:

In [97]:
df_final[['Case ID', 'Activity', 'Complete Timestamp']].head(9)

,Case ID,Activity,Complete Timestamp
0,0,Assign seriousness,2012-10-09 14:50:17
1,0,Take in charge ticket,2012-10-09 14:51:01
2,0,Take in charge ticket,2012-10-12 15:02:56
3,0,Resolve ticket,2012-10-25 11:54:26
4,0,Closed,2012-11-09 12:54:39
5,1111,Assign seriousness,2012-04-03 08:55:38
6,1111,Take in charge ticket,2012-04-03 08:55:53
7,1111,Resolve ticket,2012-04-05 09:15:52
8,1111,Closed,2012-05-19 09:00:28


In [115]:
number_cases = len(df_final['Case ID'].unique())

In [116]:
number_cases

4557

Spliting data (train, test, validation)

In [98]:
from sklearn.model_selection import train_test_split

train_cases, test_cases  = train_test_split(sorted_time_cases, test_size=0.4, 
                                            shuffle=False, 
                                            random_state=42
                                           )

test_cases, val_cases  = train_test_split(test_cases, test_size=0.5, 
                                            shuffle=False, 
                                            random_state=42
                                           )

In [99]:
len(train_cases), len(test_cases), len(val_cases)

(2748, 916, 916)

In [117]:
len(train_cases) / number_cases

0.6030283080974326

Dataframes creation

In [107]:
df_train = df_final[df_final['Case ID'].isin(train_cases)].reset_index().drop(['index'], axis=1)
df_test = df_final[df_final['Case ID'].isin(test_cases)].reset_index().drop(['index'], axis=1)
df_val = df_final[df_final['Case ID'].isin(val_cases)].reset_index().drop(['index'], axis=1)

In [118]:
df_train.shape

(12999, 4)

Overriding cases checking

In [110]:
print(
    set(test_cases).intersection(set(train_cases)),
    set(test_cases).intersection(set(val_cases)),
    set(train_cases).intersection(set(val_cases))
)

set() set() set()


In [111]:
df_sorted = pd.concat([df_train, df_val, df_test])

In [112]:
df_sorted.head()

,Case ID,Activity,Complete Timestamp,Resource
0,2222,Assign seriousness,2010-10-29 10:14:06,Value 6
1,2222,Take in charge ticket,2010-11-03 16:16:11,Value 7
2,2222,Resolve ticket,2010-11-03 16:21:17,Value 7
3,2222,Closed,2010-11-30 11:20:18,Value 5
4,3333,Assign seriousness,2010-12-15 14:31:53,Value 8


In [113]:
df_sorted.to_csv('Helpdesk2017_sorted.csv', index=False)

Statistical analisys